# CEE 697M - Final Project: Jerry Chen and Aniruddha Prasad


In [6]:
%pip install pyreadstat

Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries:

In [24]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from math import sqrt
from patsy import dmatrix

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

import statsmodels.api as sm
from statsmodels.gam.tests.test_penalized import df_autos
from statsmodels.gam.smooth_basis import BSplines
from statsmodels.gam.generalized_additive_model import GLMGam

import scipy as sp
from scipy.interpolate import interp1d





## Import the data:

In [31]:
# df = pd.read_spss("1ResearchProjectData.sav")
df = pd.read_csv("1ResearchProjectData.csv")

## Encode data into classes by hand to make applying models easier.
df = df.drop(columns=['wesson'])
df = df.dropna()

## Pre-process the data

### Replace the name of teacher with 1, 2 or 3
 Ruger = 1
 Wesson = 2
 Smith = 3
### Replace lunch status with 0 and 1
 Free lunch = 0
 Paid lunch = 1
### Replace Gender with 1's and 0's
 Female = 0
 Male = 1
### Replace ethinicity with 1 through 4
 African American = 1
 Hispanic = 2
 Caucasian = 3
 Asian = 4

In [34]:
# Teacher
df.replace('Ruger',1,inplace=True)
df.replace('Wesson',2,inplace=True)
df.replace('Smith',3,inplace=True)

# Lunch status
df.replace('Free lunch',0,inplace=True)
df.replace('Paid lunch',1,inplace=True)

# Gender:
df.replace('Female',0,inplace=True)
df.replace('Male',1,inplace=True)

#Ethnicity
df.replace('African-American',1,inplace=True)
df.replace('Hispanic',2,inplace=True)
df.replace('Caucasian',3,inplace=True)
df.replace('Asian',4,inplace=True)

df.corr()

,Student,Teacher,Gender,Ethnic,Freeredu,Score
Student,1.000000,0.458852,-0.034218,0.026850,0.002378,0.365915
Teacher,0.458852,1.000000,0.036006,0.067239,0.080995,0.355672
Gender,-0.034218,0.036006,1.000000,-0.059034,0.014503,-0.130579
Ethnic,0.026850,0.067239,-0.059034,1.000000,0.030752,-0.012077
Freeredu,0.002378,0.080995,0.014503,0.030752,1.000000,-0.044611
Score,0.365915,0.355672,-0.130579,-0.012077,-0.044611,1.000000


## Split data into Training and Validation sets:

Regression models: KNN, Spline, random forest regressor

Classification models: KNN, SVC, random forest classifier

### KNN Regression

In [35]:
X = pd.DataFrame(df, columns=['Teacher', 'Gender', 'Ethnic', 'Freeredu'])
y = pd.DataFrame(df, columns=['Score'])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

clf = KNeighborsRegressor(n_neighbors=5, p=2)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(f'accuracy: {clf.score(X_train, y_train)}')


accuracy: 0.3461882132641373


### Spline Regression

In [23]:
X = df['Score']
y = df['Teacher']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)


# 1st spline with degrees 30, 60, 90
transformed_x3 = dmatrix("bs(train, knots=(30,60,90), degree=8, include_intercept=False)", 
                {"train": X_train},return_type='dataframe')
fit1 = sm.GLM(y_train, transformed_x3).fit()

# 2nd spline with degrees 40, 50, 60
transformed_x3 = dmatrix("bs(train, knots=(40,50,60), degree=6, include_intercept=False)", 
                {"train": X_train},return_type='dataframe')
fit2 = sm.GLM(y_train, transformed_x3).fit()

# 3rd spline with degrees 50, 60, 70
transformed_x3 = dmatrix("bs(train, knots=(50,60,70), degree=3, include_intercept=False)", 
                {"train": X_train},return_type='dataframe')
fit3 = sm.GLM(y_train, transformed_x3).fit()


xp = np.linspace(X_test.min(),X_test.max(), 100)


In [17]:
prediction_1 = fit1.predict(dmatrix("bs(xp, knots=(30,60,90), degree = 3, include_intercept=False)", {"xp": X_train}, return_type='dataframe'))
prediction_2 = fit2.predict(dmatrix("bs(xp, knots=(40,50,60), degree = 6, include_intercept=False)", {"xp": X_train}, return_type='dataframe'))
prediction_3 = fit3.predict(dmatrix("bs(xp, knots=(50,60,70), degree = 8, include_intercept=False)", {"xp": X_train}, return_type='dataframe'))

sns.scatterplot(x=X_train, y=y_train)
plt.plot(X_train, prediction_1, color='orange', label='Natural spline, df=3, knots=(30,60,90)')
plt.plot(X_train, prediction_2, color='red', label='Natural spline, df=6, knots=(40,50,60)')
plt.plot(X_train, prediction_3, color='green', label='Natural spline, df=8, knots=(50,60,70)')
plt.legend()
plt.show()

fit2.summary()

ValueError: shapes (151,7) and (12,) not aligned: 7 (dim 1) != 12 (dim 0)